# Automatic Question Answering System

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityakotikalapudi","key":"ec70c8ea64fbe16babec93be1c6d07d4"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d narendrageek/mental-health-faq-for-chatbot

  0% 0.00/50.1k [00:00<?, ?B/s]
100% 50.1k/50.1k [00:00<00:00, 19.7MB/s]


In [ ]:
!unzip mental-health-faq-for-chatbot.zip

Archive:  mental-health-faq-for-chatbot.zip
  inflating: Mental_Health_FAQ.csv   


In [ ]:
import nltk,gensim
import pandas as pd
import tensorflow as tf
df=pd.read_csv("Mental_Health_FAQ.csv")
#df.columns=["Questions","Answers"]
df = df.drop(columns="Question_ID")
print(df)

                                            Questions                                            Answers
0         What does it mean to have a mental illness?  Mental illnesses are health conditions that di...
1                     Who does mental illness affect?  It is estimated that mental illness affects 1 ...
2                         What causes mental illness?  It is estimated that mental illness affects 1 ...
3   What are some of the warning signs of mental i...  Symptoms of mental health disorders vary depen...
4             Can people with mental illness recover?  When healing from mental illness, early identi...
..                                                ...                                                ...
93            How do I know if I'm drinking too much?  Sorting out if you are drinking too much can b...
94  If cannabis is dangerous, why are we legalizin...  Cannabis smoke, for example, contains cancer-c...
95       How can I convince my kids not to use drugs?  

# Tokenizing the questions

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
question = df['Questions']
print(type(question[0]))
for i in range(0,len(question)):
  print(nltk.word_tokenize(question[i]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
<class 'str'>
['What', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness', '?']
['Who', 'does', 'mental', 'illness', 'affect', '?']
['What', 'causes', 'mental', 'illness', '?']
['What', 'are', 'some', 'of', 'the', 'warning', 'signs', 'of', 'mental', 'illness', '?']
['Can', 'people', 'with', 'mental', 'illness', 'recover', '?']
['What', 'should', 'I', 'do', 'if', 'I', 'know', 'someone', 'who', 'appears', 'to', 'have', 'the', 'symptoms', 'of', 'a', 'mental', 'disorder', '?']
['How', 'can', 'I', 'find', 'a', 'mental', 'health', 'professional', 'for', 'myself', 'or', 'my', 'child', '?']
['What', 'treatment', 'options', 'are', 'available', '?']
['If', 'I', 'become', 'involved', 'in', 'tr

In [ ]:
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word) 
    #sentence=sent_stemmed
    
    return sentence
                    
def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["Questions"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["Questions"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['mean mental illness', 'mental illness affect', 'causes mental illness', 'warning signs mental illness', 'people mental illness recover', 'know appears symptoms mental disorder', 'mental health professional child', 'treatment options available', 'involved treatment need know', 'difference mental health professionals', 'mental health professional right child', 'involved treatment need know', 'help', 'know starting new medication', 'feel better taking medication mean cured stop taking', 'help paying medication', 'therapy', 'learn types mental health treatment', 'different types mental health professionals', 'support group', 'inpatient care', 'local services', 'learn information clinical trials', 'learn information creating psychiatric advance directive', 'mental health', 'support doesnt help', 'prevent mental health problems', 'cures mental health problems', 'causes mental health problems', 'im worried mental health', 'know im unwell', 'im worried friend relative', 'deal telling', 'need

In [ ]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

#from gensim import corpora

#dictionary = corpora.Dictionary(sentence_words)
#for key, value in dictionary.items():
#   print(key, ' : ', value)

def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    
    # Creating the dictionary for the unique words
    unique_word_dict = {}
    temp = 0 
    
    for i in range(0,len(text)) :
      words = list(set(text[i]))

      words.sort()
      
        
      for i,word in enumerate(words) :
            if word not in unique_word_dict.keys():
              unique_word_dict.update({
                  word: temp
              })
              temp = temp + 1
    print("\n")
    return unique_word_dict

unique_word_dict = create_unique_word_dict(sentence_words) 
print(unique_word_dict)





{'a': 0, 'does': 1, 'have': 2, 'illness': 3, 'it': 4, 'mean': 5, 'mental': 6, 'to': 7, 'what': 8, 'affect': 9, 'who': 10, 'causes': 11, 'are': 12, 'of': 13, 'signs': 14, 'some': 15, 'the': 16, 'warning': 17, 'can': 18, 'people': 19, 'recover': 20, 'with': 21, 'appears': 22, 'disorder': 23, 'do': 24, 'i': 25, 'if': 26, 'know': 27, 'should': 28, 'someone': 29, 'symptoms': 30, 'child': 31, 'find': 32, 'for': 33, 'health': 34, 'how': 35, 'my': 36, 'myself': 37, 'or': 38, 'professional': 39, 'available': 40, 'options': 41, 'treatment': 42, 'become': 43, 'in': 44, 'involved': 45, 'need': 46, 'between': 47, 'difference': 48, 'is': 49, 'professionals': 50, 'right': 51, 'else': 52, 'get': 53, 'help': 54, 'where': 55, 'before': 56, 'medication': 57, 'new': 58, 'starting': 59, 'after': 60, 'am': 61, 'and': 62, 'better': 63, 'cured': 64, 'feel': 65, 'stop': 66, 'taking': 67, 'this': 68, 'paying': 69, 'go': 70, 'therapy': 71, 'about': 72, 'learn': 73, 'types': 74, 'different': 75, 'group': 76, 's

In [ ]:
def GenerateVectors(l1:list, d:dict):
  sample = []


  for i in range(0,len(l1)):
   
    if l1[i] in d.keys():
      word_num=d.get(l1[i])
      t = (word_num,1)
      sample.append(t)
  return sample     

#for i in range(0,len(sentence_words)):
for i in sentence_words:
  print(i)
  print(GenerateVectors(i,unique_word_dict))

def convert(q:str):
  q=q.rstrip()
  m=q.split(" ")
  return m


question_orig="how to reduce stress and  depression"
question=clean_sentence(question_orig,stopwords=False)
print(convert(question))
print(GenerateVectors(convert(question),unique_word_dict))


['what', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness']
[(8, 1), (1, 1), (4, 1), (5, 1), (7, 1), (2, 1), (0, 1), (6, 1), (3, 1)]
['who', 'does', 'mental', 'illness', 'affect']
[(10, 1), (1, 1), (6, 1), (3, 1), (9, 1)]
['what', 'causes', 'mental', 'illness']
[(8, 1), (11, 1), (6, 1), (3, 1)]
['what', 'are', 'some', 'of', 'the', 'warning', 'signs', 'of', 'mental', 'illness']
[(8, 1), (12, 1), (15, 1), (13, 1), (16, 1), (17, 1), (14, 1), (13, 1), (6, 1), (3, 1)]
['can', 'people', 'with', 'mental', 'illness', 'recover']
[(18, 1), (19, 1), (21, 1), (6, 1), (3, 1), (20, 1)]
['what', 'should', 'i', 'do', 'if', 'i', 'know', 'someone', 'who', 'appears', 'to', 'have', 'the', 'symptoms', 'of', 'a', 'mental', 'disorder']
[(8, 1), (28, 1), (25, 1), (24, 1), (26, 1), (25, 1), (27, 1), (29, 1), (10, 1), (22, 1), (7, 1), (2, 1), (16, 1), (30, 1), (13, 1), (0, 1), (6, 1), (23, 1)]
['how', 'can', 'i', 'find', 'a', 'mental', 'health', 'professional', 'for', 'myself', 'or', 'my', 'child']
[

In [ ]:

from gensim import corpora
dictionary = corpora.Dictionary(sentence_words)
import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)


question_orig="Who does mental illness affect?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)

what does it mean to have a mental illness
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
who does mental illness affect
[(1, 1), (3, 1), (6, 1), (9, 1), (10, 1)]
what causes mental illness
[(3, 1), (6, 1), (8, 1), (11, 1)]
what are some of the warning signs of mental illness
[(3, 1), (6, 1), (8, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1)]
can people with mental illness recover
[(3, 1), (6, 1), (18, 1), (19, 1), (20, 1), (21, 1)]
what should i do if i know someone who appears to have the symptoms of a mental disorder
[(0, 1), (2, 1), (6, 1), (7, 1), (8, 1), (10, 1), (13, 1), (16, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]
how can i find a mental health professional for myself or my child
[(0, 1), (6, 1), (18, 1), (25, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]
what treatment options are available
[(8, 1), (12, 1), (40, 1), (41, 1), (42, 1)]
if i become involved in

In [ ]:
import numpy as np
def ListtoArray(a1:list, d:dict):
  sample_question = []

  for j in range(0,len(a1)):
    if a1[j] in d.keys():
      word_numq=d.get(a1[j])
      sample_question.append(word_numq)
  print(sample_question)
  question_array = np.array(sample_question)
  return question_array
def Listt(a1:list, d:dict):
  sample_question = []

  for j in range(0,len(a1)):
    if a1[j] in d.keys():
      word_numq=d.get(a1[j])
      sample_question.append(word_numq)
  return sample_question

def CaluculateMagnitude(m1:list, d:dict):
  la = ListtoArray(m1,d)
  magnitudeq = np.linalg.norm(la)
  
  return magnitudeq
 
k = []

def CosineSimilarity(l1:list, d:dict, q1:list, k:list):
  # for magnitude of question
  qm = CaluculateMagnitude(q1,d)

  # for magnitude of sentences
  sm = CaluculateMagnitude(l1,d)
  buckets = [] # stores the vector of dataset question
  for i in range(0,len(unique_word_dict)):
      buckets.append(0)
 
  for i in range(0,len(unique_word_dict)):
    if i in Listt(l1,d):
      buckets[i]=i
  print("vector representation",buckets)

  bucket = []
  for i in range(0,len(unique_word_dict)):
      bucket.append(0)
 
  for i in range(0,len(unique_word_dict)):
    if i in Listt(q1,d):
      bucket[i]=i
  
  b1=np.array(bucket)
  b2=np.array(buckets)


  dp= np.multiply(b1,b2)
  dp=np.sum(dp)
  dp=dp/(sm*qm)
  k.append(dp)
  print(dp)

def convert(q:str):
  q=q.rstrip()
  m=q.split(" ")
  return m


question_orig="How to reduce stress and depression"
question=clean_sentence(question_orig,stopwords=True)
q1 = convert(question)
#print(q1)
#print(GenerateVectors(convert(question),unique_word_dict))
q1_origin = q1
#print(q1_origin)
print(len(sentence_words))
print("ques ",q1)
for i in range(0,len(sentence_words)):
  
  
  print(CosineSimilarity(sentence_words[i],unique_word_dict,q1,k))
  print(sentence_words[i],"cosine value-",k[i])
print(k)
print("sentence index of maximum cosine",k.index(max(k)))
required_question_index = k.index(max(k))

p = np.array(k)
mx=np.max(p)
print(mx)

#k.clear() 

98
ques  ['reduce', 'stress', 'depression']
[214, 141]
[8, 1, 4, 5, 7, 2, 0, 6, 3]
vector representation [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.0
None
['what', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness'] cosine value- 0.0
[214, 141]
[10, 1, 6, 

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        
    
retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 0.7071067811865475 what does it mean to have a mental illness
1 0.9999999999999998 who does mental illness affect
2 0.8944271909999157 what causes mental illness
3 0.8944271909999157 what are some of the warning signs of mental illness
4 0.8944271909999157 can people with mental illness recover
5 0.7071067811865475 what should i do if i know someone who appears to have the symptoms of a mental disorder
6 0.7071067811865475 how can i find a mental health professional for myself or my child
7 0.7893522173763262 what treatment options are available
8 0.7999999999999998 if i become involved in treatment what do i need to know
9 0.8137334712067349 what is the difference between mental health professionals
10 0.7071067811865475 how can i find a mental health professional right for my child or myself
11 0.7999999999999998 if i become involved in treatment what do i need to know
12 0.7452413135250993 where else can i get help
13 0.7071067811865475 what should i know before starting a new med

In [ ]:
from gensim.models import Word2Vec 
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:            
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    
v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved glove model")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);

Loaded glove model
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved glove model


In [ ]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [ ]:

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.9063522346220526 mean mental illness
1 1.0 mental illness affect
2 0.8550026123349829 causes mental illness
3 0.700237154321032 warning signs mental illness
4 0.8104079249279282 people mental illness recover
5 0.7010398608592144 know appears symptoms mental disorder
6 0.6505212700998076 mental health professional child
7 0.19766085191988983 treatment options available
8 0.37015220174174746 involved treatment need know
9 0.6713136722860574 difference mental health professionals
10 0.6339447798140007 mental health professional right child
11 0.37015220174174746 involved treatment need know
12 0.25816243824319224 help
13 0.39991993493487465 know starting new medication
14 0.4661142601299621 feel better taking medication mean cured stop taking
15 0.3676493890035998 help paying medication
16 0.17757058450747093 therapy
17 0.6416267226339917 learn types mental health treatment
18 0.6006161343734778 different types mental health professionals
19 0.1120924045403334 support group
20 0.33990

In [ ]:
#With Glove

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model));
    
question_embedding=getPhraseEmbedding(question,glove_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.9408499504676836 mean mental illness
1 1.0000000000000002 mental illness affect
2 0.9682996172048146 causes mental illness
3 0.9455110787855933 warning signs mental illness
4 0.9590149965641712 people mental illness recover
5 0.9358827525265986 know appears symptoms mental disorder
6 0.8688163153042198 mental health professional child
7 0.7484632052149842 treatment options available
8 0.7983911871031923 involved treatment need know
9 0.906645339566196 difference mental health professionals
10 0.8586648601303251 mental health professional right child
11 0.7983911871031923 involved treatment need know
12 0.7191278685628064 help
13 0.7359511446135815 know starting new medication
14 0.7743642099181619 feel better taking medication mean cured stop taking
15 0.8198821319884015 help paying medication
16 0.7218247022314166 therapy
17 0.9223850029535061 learn types mental health treatment
18 0.8597431726057122 different types mental health professionals
19 0.6355581258247984 support group
2

## ANSWER EXTRACTION

In [ ]:
ans =df["Answers"]
print(ans[required_question_index])
required_ans = ans[required_question_index]

required_ans = required_ans.split(".")
def convert(q:str):
  q=q.rstrip()
  m=q.split(" ")
  return m
list_ans = []
for i in required_ans:
    i = i.lower().strip()
    i = clean_sentence(i,stopwords=True)  
    print(i)
    list_ans.append(i)
print(list_ans)

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
#question = df['Questions']
print(type(list_ans[0]))
from nltk import pos_tag

stem_sentence = []
for i in range(0,len(list_ans)):
  words = nltk.word_tokenize(list_ans[i])
  print(words)
  stem = [porter.stem(word) for word in words]
  pos = nltk.pos_tag(words)
  #print("pos",pos)
  stem_sentence.append(stem)
  #print("stem",stem)

ans_dict = create_unique_word_dict(stem_sentence)
print("ans ",ans_dict)
k1 = []
print("ques ",q1)
q1_stem = [porter.stem(word) for word in q1_origin]
print("stemmed q1 ",q1_stem)
for i in range(0,len(stem_sentence)):
  
  print("sent ",stem_sentence[i])
  

  print(CosineSimilarity(stem_sentence[i],ans_dict,q1_stem,k1))
  print(stem_sentence[i],"cosine value-",k1[i])
print(k1)
print("sentence index of maximum cosine",k1.index(max(k1)))
required_answer_index = k1.index(max(k1))

p_ans = np.array(k1)
mx_ans=np.max(p_ans)
print(" ")
for i in range(0,len(required_ans)):

  print(" ")
  print("sentence",i+1,")",required_ans[i])
print(" ")
print("Question-",question_orig)
print(" ")
print("recquired sentence-",list_ans[required_answer_index])
print(" ")
print("stem-",stem_sentence[required_answer_index])
print(" ")
print("max cosine value-",mx_ans)



Stress and anxiety are often used interchangeably, and there is overlap between stress and anxiety. Stress is related to the same ‘fight, flight, or freeze’ response as anxiety, and the physical sensations of anxiety and stress may be very similar. 
 The cause of stress and anxiety are usually different, however. Stress focuses on mainly external pressures on us that we’re finding hard to cope with. When we are stressed, we usually know what we’re stressed about, and the symptoms of stress typically disappear after the stressful situation is over. 
 Anxiety, on the other hand, isn’t always as easy to figure out. Anxiety focuses on worries or fears about things that could threaten us, as well as anxiety about the anxiety itself. Stress and anxiety are both part of being human, but both can be problems if they last for a long time or have an impact on our well-being or daily life.
stress anxiety interchangeably overlap stress anxiety
stress related fight flight freeze response anxiety ph

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide
